In [19]:
import pandas as pd
import numpy as np
import time
import re as re
from selenium import webdriver
import translators as ts
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()
def get_data_salling(url):
    
    
    """
    This function will collect the grant information provided from the Salling foundation grant database with the url: https://www.sallingfondene.dk/donationer
    
    The paths are found using devtools, and found through the webpage.
    
    note: When the window opens, one must actively choose a web browser (eg. Ecosia) and close the
    the Cookie popup window - to start the extraction. 
    
    The information is saved in the lists of the following groups:
    
    project_category_l = []
    title_l = []
    year_l = []
    desc_l = []
    amount_l = []
    
    and if data is not avaiable for a certain group, then "NA" is inserted. 
    
    The lists are finally zipped together in the biglist dataframe.
    
    input: target_url (https://www.sallingfondene.dk/donationer")
    return: Dataframe with text extracted
    """
    
    project_category_l = []
    title_l = []
    year_l = []
    desc_l = []
    amount_l = []

    
    
    
    driver.get(url)
    driver.maximize_window() #opening full web browser window
    time.sleep(5)
    
    #hradcoded the first grant because it's structure is different from the rest
    
    ActionChains(driver).move_to_element(driver.find_element(By.XPATH, f"/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[2]/article[1]/div[2]/header/h3/a")).perform()  #move to place
    time.sleep(5)
    
    
    
    for i in range(1,208): #208

        try:
            if i % 8 == 0:
                ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[3]/button")).perform()  #move to place
                element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[3]/button")))
                element.click()
                time.sleep(5)
            
            ActionChains(driver).move_to_element(driver.find_element(By.XPATH, f"/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[2]/article[{i}]/div[2]/header/h3/a")).perform()  #move to place
            time.sleep(5)
            
            title_receiver = driver.find_element(By.XPATH, f"/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[2]/article[{i}]/div[2]/header/h3/a").text
            project_category = driver.find_element(By.XPATH, f"/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[2]/article[{i}]/div[2]/header/span/span").text
            amount = driver.find_element(By.XPATH, f"/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[2]/article[{i}]/div[2]/div/div[1]").text
            year = driver.find_element(By.XPATH, f"/html/body/div[2]/div/main/div/div/div/div[2]/div[2]/div[2]/article[{i}]/div[2]/div/div[2]").text
            
         
            
            project_category_l.append(project_category)
            title_l.append(title_receiver)
            amount_l.append(amount)
            year_l.append(year)
            
            
        except NoSuchElementException: #avoid programme crashing
            print("Loop ended")
            print(i)
            print(title_receiver)

 

    df_salling = pd.DataFrame(list(zip(year_l, title_l, amount_l, project_category_l)), columns=["Year",'Title and Reciever','Grant size (DKK)', "Project Category"])

    return df_salling

biglist = get_data_salling("https://www.sallingfondene.dk/donationer")
pd.set_option('display.max_columns', None)
biglist.to_csv('salling_data.csv', index=True)